In [ ]:
# Running in Google Colab? Run this cell
!wget https://raw.githubusercontent.com/CIERA-Northwestern/REACHpy/main/Module_3/data/binint_data.txt

# If you're not running in Colab, this file should be in the data directory.
# Change the loading path of the file to include 'data/' when the file is loaded

# Challenge Problem 3: Parsing Binary Interaction Data

There are many cases where computer software will output some data in a human readable format, instead of a machine readable format like the `.csv` files you've encountered in this course. In those cases, it is impossible to use functions like `np.loadtxt()` or `pd.read_csv` to parse the data. Instead, we require a judicious choice of the string parsing tools of python to get our data in a more machine-friendly format. Here, we encounter one example of this.

The data for this exercise comes large scale simulations of globular clusters, star clusters which are very old and have up to millions of stars. In these environments, stars can pair up and form tight binary systems. When these tight binary systems encounter another single star or binary, figuring out the result of the gravitational encounter requires a high precision simulation, so this smaller simulation is quickly done and then the results of the interaction are put back into the larger simulation. This file contains a large number of these results for part of a single simulation of a globular cluster. Due to the size of the output, it was necessary to truncate the file. If you would like to see the full version of these results, you can download the cluster simulation results [here](https://cmc.ciera.northwestern.edu) using the options `N = 1.6e6`, `rv = 1`, `rg = 2`, and `Z = 0.0002` and looking at the `initial.binint.log` file. 

Now that you've downloaded the file, let's see the first two interactions in the output.

```
********************************************************************************
type=BS t=0.00143809484
params: b=1.75682 v=2.07854
input: type=single m=2.4765 R=1.13746 Eint=0 id=200248 ktype=1
input: type=binary m0=86.0998 m1=0.81708 R0=0.00182532 R1=0.717717 Eint1=0 Eint2=0 id0=1261439 id1=145163 a=27.7383 e=0.620279 ktype1=14 ktype2=1
status: DE/E=-1.22029e-08 DE=-1.66964e-09 DL/L=2.13858e-09 DL=6.82604e-10 DE_GW/E=0 DE_GW=0 v_esc_cluster[km/s]=118.521 tcpu=0.02
outcome: nstar=3 nobj=2:  0 [2 1] (single-binary)
output: type=single m=2.4765 R=1.13746 Eint=0 id=200248 ktype=1
output: type=binary m0=0.81708 m1=86.0998 R0=0.717717 R1=0.00182532 Eint1=0 Eint2=0 id0=145163 id1=1261439 a=27.6085 e=0.615538 ktype1=1 ktype2=14
********************************************************************************
type=BS t=0.00143809484
params: b=3.37835 v=0.681853
input: type=single m=0.213453 R=0.224568 Eint=0 id=1924 ktype=0
input: type=binary m0=85.2106 m1=0.229002 R0=33.6832 R1=0.236034 Eint1=0 Eint2=0 id0=24087 id1=534504 a=30.3525 e=0.470706 ktype1=1 ktype2=0
status: DE/E=1.11829e-08 DE=-6.95098e-12 DL/L=2.92849e-10 DL=2.03801e-12 DE_GW/E=-0 DE_GW=0 v_esc_cluster[km/s]=118.183 tcpu=0.01
outcome: nstar=3 nobj=2:  0 [1 2] (single-binary)
output: type=single m=0.213453 R=0.224568 Eint=0 id=1924 ktype=0
output: type=binary m0=85.2106 m1=0.229002 R0=33.6832 R1=0.236034 Eint1=0 Eint2=0 id0=24087 id1=534504 a=30.4538 e=0.467267 ktype1=1 ktype2=0
********************************************************************************
```

First, notice that each separate interaction is separated by a fixed length string of `*` symbols. We can use this to our advantage later. Next, let's go through each line and I'll highlight the most relevant quantities. Note that in almost all cases, the discussion concerning complicated nonstandard units is not of immediate importance and are simply there for your benefit in case you're curious.
1. Line 1
    * `type`: This refers to the type of the interaction. Every interaction will either be type `BS` for a binary object and a single object or `BB` for two binary objects.
    * `t`: This refers to the time in the simulation that this interaction takes place. The system of units is a little strange, but for this specific simulation, you can get the time in Myr by multiplying by the conversion factor `timeunitsmyr=2501.66`. If you download the full simulation data, this and other conversion factors can be found in the `initial.conv.sh` file.
2. Params
    * `b`: This is a number that refers to how close the two inputs would pass by each other if there were no force deflecting their path. In this file, the units of `b` are either `a` for `BS` interactions or `a1 + a2` for `BB` interactions (see below).
    * `v`: This is the relative velocity between the two inputs in units of something called the critical velocity. For explanation, see equations 1 and 2 in [Fregeau et al. (2004)](https://articles.adsabs.harvard.edu/pdf/2004MNRAS.352....1F).
3. Input
    * `type`: As stated before, the type of a body can either be a single or a binary. Depending on the type, the amount of information in these lines will change. If the type is a binary, some quantities will have a suffix of `0` or `1`.
    * `m`: The mass of the object in solar masses.
    * `R`: The radius of the object in solar radii.
    * `Eint`: Ignore this for now.
    * `id`: The id of the object. You can choose to keep or ignore this.
    * `ktype`: The type of star, using the [BSE k types](https://cosmic-popsynth.github.io/docs/stable/output_info/index.html).
    * `a`: The semimajor axis of the binary in AU.
    * `e`: The eccentricity of the binary.
4. Status
    * `DE/E`: Fractional energy lost.
    * `DL/L`: Fractional angular momentum lost. 
    * `DE_GW/E`: Fractional energy lost due to gravitational waves.
    * `v_esc_cluster[km/s]`: Escape speed from the cluster.
    * `t_cpu`: Time for the simulation to complete in seconds.
5. Outcome:
    * `nstar`: Number of stars left at the end. This can be less than the number you begin with if two of the stars collide during the encounter.
    * `nobj`: Number of objects left. These could be singles, binaries, or triples.
    * `i [j k]`: A string of numbers and symbols showing the final configuration. You can skip this if you'd like.
    * `outcome`: A string version of the above. Technically, you can also skip this since we have the below.
6. Output:
    * `type`: Same types as before, but with one additional type: `triple`. For the triple type, there is an inner binary orbited by a third object. In this case, the objects in the inner binary will have suffixes `in0` and `in1` and the outer orbit/object will have the suffix `out`. For example:
    ```
    output: type=triple min0=39.1967 min1=33.0416 mout=43.4431 Rin0=11.4836 Rin1=8.86084 Rout=0.000921734 Eintin0=0 Eintin1=0 Eintout=-1.47613e+48 idin1=209143 idin2=1604749 idout=1614816:1706917 ain=0.29791 aout=149.748 ein=0.00447152 eout=0.943376 ktypein1=1 ktypein1=1 ktypeout=14
    ```

Possible outcomes for `BS` encounters:
* binary single
* single single single
* binary
* single single
* single
* and permutations of the above

Possible outcomes for `BB` encounters:
* binary binary
* binary single single
* single single single single
* triple single
* binary single
* single single single
* triple
* binary
* single single
* single
* and permutations of the above

Your first task is to come up with a logical way of storing this data in a tabular format. There are two broad options. You can have a single large table for all the interactions and have some rows have empty columns depending on the output of the interaction. Or, you can split the data into multiple tables depending on both the inputs and the outputs. If you go with the second option, it might be useful to have one big table that contains the parameters of all the interactions (e.g., the time of the interaction, the index, the type, and the outcome) for easy tracking.

After you've done your parsing, it's time for data analysis. Here's some questions to get you started:
1.   How many total interactions are recorded in this file? How many are `BS` vs `BB`? Over what timeframe (in Myr) do these interactions take place?
2.   How many interactions involve at least one black hole? How many interactions are composed of only black holes? How does the number of black hole interactions evolve over time?
3.   In binary-single interactions that end with a binary-single, we like to distinguish between exchanges and preservations. For how many binary-single interactions with the binary-single outcome are the members of the binary the original members of the binary? For when an exchange occurs, what can you say about which stars end up in the binary? Does this conclusion also seem to apply to the preservations too?
4.   How many interactions result in one or more collisions? How does the number of collisions relate to the different parameters of the interaction (e.g. `a`, `b`, `v`, etc.)? What stellar types are most commonly involved in a collision?

Make sure to use plots to answer these questions.

Optional tasks if you have time:
*   Now, make some histograms of some of the physical values, including exoplanet radius, semimajor axis, host star mass, etc. For your histogram of the exoplanet mass, restrict yourself to the rows where `pl_bmassprov=='Mass'` instead of `Msini`. Do you notice anything interesting? For example, you should find that most host stars will have masses around 1 solar mass. This is because most exoplanet surveys have purposefully targeted stars similar to our own Sun. Other things to look into: Are there more planets further away than close to us? Does planetary radius or mass seem to peak at any specific values?
*   Kepler derived his third law from actual observations of the solar system planets where he plotted the semimajor axis vs the orbital period. Try doing this with log-log axes. When you do this for the solar system planets, you get a straight line. What do you notice in this case?
*   Is there any correlation between distance from the star and planetary mass or radius? Make some scatter plots to investigate this.
*   From the planetary mass and radius, you can get the planetary density. Make a histogram of this. What do you notice?
*   Anything else you can think of! Try stuff out.

Keep in mind that some of the conclusions you draw here might not be true. Observations are prone to some level of bias, also known as "selection effects". For example, it's very hard to directly image exoplanets that are small and close to their host star, so the method of direct imaging is biased towards larger exoplanets and larger distances. Each detection method is biased in some way, so take some of your conclusions with a grain of salt.